<a href="https://colab.research.google.com/github/maddyy13/animal-sound/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

data = pd.read_csv('/CSV.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-08-15,40.500000,41.509998,40.200001,41.029999,40.373474,175100
2022-08-16,40.580002,41.389999,40.410000,40.630001,39.979877,124700
2022-08-17,40.400002,40.910000,40.029999,40.630001,39.979877,127200
2022-08-18,40.369999,40.840000,40.049999,40.410000,39.763393,69600
2022-08-19,40.150002,40.680000,39.689999,40.180000,39.537075,118500
...,...,...,...,...,...,...
2023-08-09,31.120001,31.770000,31.049999,31.549999,31.549999,48900
2023-08-10,31.730000,31.799999,30.719999,30.959999,30.959999,42600
2023-08-11,31.080000,31.320000,30.340000,30.370001,30.370001,53100


In [16]:
# Normalize data
scaler = MinMaxScaler()
data['Close'] = scaler.fit_transform(data[['Close']])
scaler

MinMaxScaler()

In [15]:
# Convert the data to supervised learning format
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data)-look_back):
        X.append(data[i:(i+look_back)])
        y.append(data[i+look_back])
    return np.array(X), np.array(y)

look_back = 20  # Number of previous time steps to use for prediction
X, y = create_dataset(data['Close'].values, look_back)
X,y

(array([[1.        , 0.97731151, 0.97731151, ..., 0.65229737, 0.68349414,
         0.73057287],
        [0.97731151, 0.97731151, 0.96483273, ..., 0.68349414, 0.73057287,
         0.65910384],
        [0.97731151, 0.96483273, 0.95178678, ..., 0.73057287, 0.65910384,
         0.633012  ],
        ...,
        [0.60238234, 0.5859331 , 0.57118557, ..., 0.43675556, 0.46228017,
         0.42881449],
        [0.5859331 , 0.57118557, 0.56608063, ..., 0.46228017, 0.42881449,
         0.39534892],
        [0.57118557, 0.56608063, 0.59897899, ..., 0.42881449, 0.39534892,
         0.39364721]]),
 array([0.65910384, 0.633012  , 0.62450372, 0.60635284, 0.58876912,
        0.54849691, 0.52467399, 0.47475896, 0.43051619, 0.44299486,
        0.46398188, 0.53545097, 0.49461143, 0.49688035, 0.57685766,
        0.61032335, 0.57799209, 0.56211013, 0.50085085, 0.53204762,
        0.50992623, 0.47589345, 0.45887694, 0.42768017, 0.4702213 ,
        0.51616566, 0.45774245, 0.41747024, 0.4134998 , 0.41747024,
 

In [49]:
# Split data into training and testing sets
train_size = int(0.8 * len(data))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(X_train, X_test)
print(y_train, y_test)

[[1.         0.97731151 0.97731151 ... 0.65229737 0.68349414 0.73057287]
 [0.97731151 0.97731151 0.96483273 ... 0.68349414 0.73057287 0.65910384]
 [0.97731151 0.96483273 0.95178678 ... 0.73057287 0.65910384 0.633012  ]
 ...
 [0.17810557 0.15655134 0.11854794 ... 0.21157114 0.226886   0.2229155 ]
 [0.15655134 0.11854794 0.19228594 ... 0.226886   0.2229155  0.26318782]
 [0.11854794 0.19228594 0.15428254 ... 0.2229155  0.26318782 0.58763463]] [[0.19228594 0.15428254 0.21781056 0.28190592 0.25411227 0.22575152
  0.25354516 0.23085645 0.21894499 0.24106638 0.24163359 0.23199094
  0.23142372 0.22915492 0.21157114 0.226886   0.2229155  0.26318782
  0.58763463 0.51446407]
 [0.15428254 0.21781056 0.28190592 0.25411227 0.22575152 0.25354516
  0.23085645 0.21894499 0.24106638 0.24163359 0.23199094 0.23142372
  0.22915492 0.21157114 0.226886   0.2229155  0.26318782 0.58763463
  0.51446407 0.55303474]
 [0.21781056 0.28190592 0.25411227 0.22575152 0.25354516 0.23085645
  0.21894499 0.24106638 0.2416

In [52]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50
7/7 [==============================] - 5s 24ms/step - loss: 0.0856
Epoch 2/50
7/7 [==============================] - 0s 23ms/step - loss: 0.0213
Epoch 3/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0128
Epoch 4/50
7/7 [==============================] - 0s 26ms/step - loss: 0.0136
Epoch 5/50
7/7 [==============================] - 0s 25ms/step - loss: 0.0101
Epoch 6/50
7/7 [==============================] - 0s 26ms/step - loss: 0.0106
Epoch 7/50
7/7 [==============================] - 0s 24ms/step - loss: 0.0095
Epoch 8/50
7/7 [==============================] - 0s 26ms/step - loss: 0.0092
Epoch 9/50
7/7 [==============================] - 0s 24ms/step - loss: 0.0092
Epoch 10/50
7/7 [==============================] - 0s 24ms/step - loss: 0.0089
Epoch 11/50
7/7 [==============================] - 0s 26ms/step - loss: 0.0085
Epoch 12/50
7/7 [==============================] - 0s 26ms/step - loss: 0.0082
Epoch 13/50
7/7 [==============================] - 0s 37ms/st

In [41]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
print(train_predict)
print(test_predict)

1/1 [==============================] - 0s 21ms/step
[[1.         0.9773115  0.9773115  ... 0.6522974  0.68349415 0.7305729 ]
 [0.9773115  0.9773115  0.9648327  ... 0.68349415 0.7305729  0.6591038 ]
 [0.9773115  0.9648327  0.95178676 ... 0.7305729  0.6591038  0.633012  ]
 ...
 [0.17810556 0.15655135 0.11854794 ... 0.21157114 0.226886   0.2229155 ]
 [0.15655135 0.11854794 0.19228594 ... 0.226886   0.2229155  0.26318783]
 [0.11854794 0.19228594 0.15428254 ... 0.2229155  0.26318783 0.5876346 ]]
[[0.19228594 0.15428254 0.21781056 0.28190592 0.25411227 0.22575152
  0.25354517 0.23085645 0.218945   0.24106638 0.2416336  0.23199093
  0.23142372 0.22915491 0.21157114 0.226886   0.2229155  0.26318783
  0.5876346  0.5144641 ]
 [0.15428254 0.21781056 0.28190592 0.25411227 0.22575152 0.25354517
  0.23085645 0.218945   0.24106638 0.2416336  0.23199093 0.23142372
  0.22915491 0.21157114 0.226886   0.2229155  0.26318783 0.5876346
  0.5144641  0.5530347 ]
 [0.21781056 0.28190592 0.25411227 0.22575152 0

In [32]:
# Calculate RMSE
train_score = np.sqrt(mean_squared_error(y_train[0], train_predict[:, 0]))
print('Train RMSE:', train_score)
test_score = np.sqrt(mean_squared_error(y_test[0], test_predict[:, 0]))
print('Test RMSE:', test_score)

Train RMSE: 0.21360990084792952
Test RMSE: 0.27729027293856046
